#### 제주도 핫플레이스 인스타그램에서 크롤링하기

##### matplotlib 한글설정

##### 셀레니움 설정

In [2]:
import matplotlib
# from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc, rcParams
import seaborn as sns

font_path = 'C:/Windows/Fonts/NanumGothic.ttf'
font = font_manager.FontProperties(fname=font_path).get_name() # 'NanumGothic'
rc('font', family=font)
rcParams['axes.unicode_minus'] = False 

In [7]:
# 셀레니움
# 모듈 임포트
from selenium import webdriver

driver = webdriver.Chrome()
url = 'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91/'
driver.get(url)

##### 첫번째 게시글 찾아서 클릭하기


In [10]:
from selenium.webdriver.common.by import By
import time

def select_first_post(driver):
    first = driver.find_elements(By.CLASS_NAME, '_aagw')[0]
    first.click()
    time.sleep(3)

select_first_post(driver)

##### 게시글 정보 가져오기

In [11]:
import re # RegularExpression 정규식 처리 모듈
from bs4 import BeautifulSoup
import unicodedata  # 유니코드 데이터 처리 모듈

def get_content(driver):
    # 1. 데이터가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 2. 본문내용 찾아오기
    try:
        content = soup.select('div._a9zr')[0].text   # _a9zr은 추후 변경됨(인스타그램 정책)
    except:
        content = ''

    # 3. 본문내용에서 해시태그 가져오기(정규식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content)

    # 4. 작성일자 정보
    try:
        date = soup.select('time._aaqe')[0]['datetime'][:10] # _aaqe은 추후 변경됨(인스타그램 정책)
    except:
        date = ''

    # 5. 좋아요수 가져오기
    try:
        # x193iq5w등은 추후 변경됨(인스타그램 정책)
        like = int(soup.select('span.x193iq5w.xeuugli.x1fj9vlw.x13faqbe.x1vvkbs.xt0psk2.x1i0vuye.xvs91rp > span')[0].text)
    except:
        like = 0

    # 6. 위치정보
    # 정규식 학습필수
    # _aagv 마지막 인덱스를 찾으면 된다
    # imgs = soup.select('div._aatk._aatl._aatm div._aagv > img')[0]['alt']
    address = ''
    try:
        regex = r'(\w+[시,읍]\s*)?' \
                r'?(\w+\d*\w*[로,리,길]\s*)' \
                r'?(\w*\d+-?\d*)?'

        place = re.findall(regex, content)
        for pl in place:
            if pl[0] != '' and pl[1] != '' and pl[2] != '':
                address = f'{pl[0]}{pl[1]}{pl[2]}'
    except:
        address = ''

    # 7. 정보 저장
    data = [content, date, like, address, tags]
    return data

get_content(driver)

['dupl._ssunnys(리그램) @aewolzzim제주애월 핫플레이스 "애월찜"제주현지인들도 항상 찾아가는 비밀 맛집❗❗ 여행객들도 꼭 가야될 필수코스❗❗좋은 기운 받고 가실분 여기여기 모여라 ❗애월해안도로에 위치한 분위기 깡패 연인 데이트 코스로도 찰떡❗❗매콤한 전복+낙지+ 갈비찜(해물찜)은 술 좋아하는 사람에게는 최고의 안주에 오션뷰는 덤❗❗❗⠀⠀🧒 애월갔으면 무조건 들렀다오자⠀⠀🏠 제주 제주시 애월읍 애월해안로 454-5📞 예약문의 0507-1403-7496..#애월맛집#제주애월맛집#애월해물찜#애월해안도로#애월가볼만한곳#애월해안도로맛집#하귀맛집#제주데이트코스#제주도민맛집#제주현지인맛집#애월맛집추천#제주도맛집#제주시맛집#제주도민맛집추천13시간',
 '2023-07-11',
 0,
 '애월읍 애월해안로 454-5',
 ['#애월맛집',
  '#제주애월맛집',
  '#애월해물찜',
  '#애월해안도로',
  '#애월가볼만한곳',
  '#애월해안도로맛집',
  '#하귀맛집',
  '#제주데이트코스',
  '#제주도민맛집',
  '#제주현지인맛집',
  '#애월맛집추천',
  '#제주도맛집',
  '#제주시맛집',
  '#제주도민맛집추천13시간']]

In [12]:
def move_next(driver):
    right = driver.find_element(By.CSS_SELECTOR, ('._aaqg._aaqh'))
    right.find_element(By.XPATH, '//button')
    right.click()
    time.sleep(3)

move_next(driver)

##### 자동 크롤링 구현

In [13]:
# 모듈 임포트 생략

driver = webdriver.Chrome()
driver.get('https://www.instagram.com')
time.sleep(2.0)

In [14]:
# 아이디, 패스워드 입력해서 자동 로그인하기
id = '01036825261'
input_id = driver.find_elements(By.CSS_SELECTOR, '._aa4b._add6._ac4d')[0]
input_id.clear()
input_id.send_keys(id)
password = 'cdt92f278*'
input_pw = driver.find_elements(By.CSS_SELECTOR, '._aa4b._add6._ac4d')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()
time.sleep(2.0)

In [15]:
# 알림설정 버튼 클릭
alarm_btn = driver.find_element(By.CSS_SELECTOR, '._a9--._a9_0')
alarm_btn.click()

In [16]:
# 제주도맛집 찾기 함수
def insta_searching(word):
    url = f'https://www.instagram.com/explore/tags/{word}'
    return url

In [17]:
# 크롤링 시작하기
word = '제주도맛집'
url = insta_searching(word)
driver.get(url)
time.sleep(10.0)

select_first_post(driver)

results = []

target = 3
count = 0
for i in range(target):
    count += 1
    try:
        data = get_content(driver)
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2.0)
        move_next(driver)
    print(count)

print(len(results))

1
2
3
4
5
6
7
8
9
10
10


In [18]:
import pandas as pd
df = pd.DataFrame(results)
df

,0,1,2,3,4
0,"dupl._ssunnys(리그램) @aewolzzim제주애월 핫플레이스 ""애월찜""제...",2023-07-11,0,애월읍 애월해안로 454-5,"[#애월맛집, #제주애월맛집, #애월해물찜, #애월해안도로, #애월가볼만한곳, #애..."
1,dusenvldhtl0291대게! 킹크랩! 홍게까지! 삼위일체 바다 보면서 제대로 ...,2023-07-11,0,제주시 서해안로 638,"[#제주도맛집, #제주맛집, #제주도횟집, #제주도대게맛집, #제주도대게, #제주도..."
2,kbk0616귀한 사진 감사합니다🫡뱅기✈️타고 제주도에? 언제?역시 멋짐👍.#Rep...,2023-07-10,800,,"[#Repost, #숙성도, #마이리틀히어로, #임영웅, #오늘도숙성도하세요., #..."
3,_or_a._nkin_t(광고)제주도에서 쓰끼다시가 가장 잘나오고 서귀포에서 오션뷰...,2023-07-11,0,서귀포시 대포로 16111,"[#서귀포맛집, #제주중문맛집, #서귀포횟집, #중문횟집, #제주도횟집, #중문맛집..."
4,lautalia_76(협찬) @cafe_the_light_jeju💗제주도에서 가장 ...,2023-07-11,1054,성산읍 한도로 269,"[#제주여행, #제주카페, #성산일출봉카페, #제주카페추천, #제주루프탑카페, #제..."
5,"hhh_e_07(리그램) @aewolzzim제주애월 핫플레이스 ""애월찜""제주현지인들...",2023-07-11,154,애월읍 애월해안로 454-5,"[#애월맛집, #제주애월맛집, #애월해물찜, #애월해안도로, #애월가볼만한곳, #애..."
6,foofilll공항근처 흑돼지집은 여기로 가세요. 성공보장입니다.제주도 #공항그때그...,2023-07-07,4075,제주시 서해안로 352-1,"[#공항그때그집제주도=흑돼지는, #푸필_제주, #제주도여행, #제주도맛집, #제주맛..."
7,matkook_집 나간 입맛도 돌아오게 만드는 물회. 싱싱한 해산물 배 터지게 먹을...,2023-07-11,990,애월읍 하귀3길 22,"[#제주, #애월밤바다📃, #맛국_제주17시간]"
8,jeju_yeongmoks23.07.10.초복을 앞두고 있는 여유가득한 제주의 오늘...,2023-07-10,1140,,"[#영목이네, #제주갈치맛집영목이네1일]"
9,hohohaha909카레가 귯귯👍 애월바다까지 담은애월 예쁜 카레맛집 🍛 😜#jej...,2023-07-12,0,,"[#jeju, #애월핫플, #구엄맛집, #애월맛집, #제주공항맛집, #제주카레추천,..."


In [19]:
import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['content','data','like','place','tags']
results_df.to_excel('./1_crawling_jejudoHotplaces.xlsx', index=False)